### Imports

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

import langchain

# LLM
from langchain_community.llms import Ollama

# Document loader
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embedding
from langchain_community.embeddings.ollama import OllamaEmbeddings

# Vector store
from langchain.vectorstores.chroma import Chroma

# Prompt
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

MODEL = 'llama3.1'
DATA_PATH = 'data'
CHROMA_PATH = 'chroma'

In [2]:
model = Ollama(model = MODEL)
# model.invoke('Olá, tudo bem?')

embeddings = OllamaEmbeddings(model = 'nomic-embed-text')

### Reading and chunking the document

In [3]:
loader = PyPDFDirectoryLoader(DATA_PATH)
documents = loader.load()
        
text_spitter = RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 80,
    length_function = len,
    is_separator_regex = False
)

chunks = text_spitter.split_documents(documents)

### Creating the Vector DB

In [4]:
db = Chroma.from_documents(
    documents = chunks,    
    embedding = embeddings,
    persist_directory = CHROMA_PATH
)

### Using LLama with RAG

In [5]:
prompt_template = """
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""

PROMPT = PromptTemplate(
    template = prompt_template, input_variables = ["summaries", "question"]
)

langchain.verbose = False

chain = RetrievalQA.from_chain_type(
    llm = model,
    retriever = db.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={
        'prompt': PROMPT,
        'document_variable_name': 'summaries'
    }
)

In [6]:
QUERY = 'Quais são as normas de saúde e seguraça do trabalho?'
response = chain.invoke(QUERY)

In [7]:
resposta = response['result']

print(f'Pergunta: {QUERY}')
print('\n')
print(f'Resposta: {resposta}')

Pergunta: Quais são as normas de saúde e seguraça do trabalho?


Resposta: As normas de saúde e segurança do trabalho adotadas pela TARRAF incluem:

• Agir conforme as normas de Saúde e Segurança do Trabalho;
• Uso e conservação adequados dos equipamentos de proteção coletiva e individual;
• Paralização da atividade em situações de risco;
• Zelar por um local de trabalho limpo e realizar descartes corretamente.

(Estas normas estão explicitadas no Código de Ética e Conduta 16, sob o título "MEIO AMBIENTE E SEGURANÇA")

SOURCES:
- Código de Ética e Conduta 16, MEIO AMBIENTE E SEGURANÇA (TARRAF)
- Código de Ética e Conduta 16, MEIO AMBIENTE E SEGURANÇA (TARRAF)
